In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
#from langchain_community.llms import Ollama
#from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
#from langchain_ollama import OllamaEmbeddings
import os
from langgraph.prebuilt import ToolNode
from typing import Literal
#from langgraph.graph import END
from langgraph.graph import START, END
from langgraph.graph import MessagesState, StateGraph


In [ ]:
# model cell

embeddings = OllamaEmbeddings(
    model="bge-m3"
)

llm = ChatOllama(
    model="qwen3:4b"
)

C:\Users\worb1\AppData\Local\Temp\ipykernel_20164\122708114.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [ ]:
# 1. 문서 로드
loader = TextLoader("./docs/RFP_requirements.md", encoding="utf-8")
documents = loader.load()

# 2. 문서 나누기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

# 3. 벡터 스토어 생성
persist_directory = "./chroma_db"
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_directory)


✅ Chroma DB에 41개의 문서를 임베딩하여 저장 완료.


In [6]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "로그인 기능에 필요한 요소는 무엇인가요?"

result = retriever.invoke(query)

print(f"✅ 검색 결과: {result}")

✅ 검색 결과: [Document(id='5512e0cb-b514-4c1c-b7c4-b91cc93bb21d', metadata={'source': './docs/RFP_requirements.md'}, page_content='- **요구사항ID**: SFR-014\n- **유형**: 기능 요구사항\n- **대분류**: 구축 LMS (Learning Management System)\n- **중분류**: 대한체육회 체육인재개발원 포털 구축\n- **소분류**:\n  - **기본정보 조회**\n    - 인사말, 연혁, 교육비전, 교육일정, 부서안내, 찾아오시는 길 등 정보 조회 :contentReference[oaicite:0]{index=0}&#8203;:contentReference[oaicite:1]{index=1}\n  - **교육안내**\n    - 교육신청절차, 수료기준, 준수사항, 연간교육계획, 교육훈련근거 등 정보 조회 :contentReference[oaicite:2]{index=2}&#8203;:contentReference[oaicite:3]{index=3}\n  - **교육신청**\n    - 과정별 안내 및 교육 검색 기능 제공\n    - 집합교육·온라인교육 구분 조회 및 신청 기능 제공\n    - 대상(선수·지도자·심판·행정가·일반체육인)별 정보 제공\n    - 분야별(스포츠·어학·소양·자격) 교육 정보 제공\n    - 심판·지도자 강습회 정보 제공 :contentReference[oaicite:4]{index=4}&#8203;:contentReference[oaicite:5]{index=5}\n  - **학습자 마이페이지 기능**\n    - 출결조회·진도율 등 진행 중 교육 과정 현황 조회\n    - 교육이력·경기인 등록 이력 등 이력 정보 제공\n    - 이수증 발급·교재 다운로드 등 편의 기능 제공\n    - 만족도 평가 기능 및 Q&A 제공\n    - 관심분야 등록·수정 및 맞춤형 교육 추천 기능\n    - 수

In [ ]:




if os.path.exists('./fastmcp-server/requirments_collection') and len(os.listdir('./fastmcp-server/requirments_collection')) > 0:
    vector_store = Chroma(
        embedding_function=embeddings,
        collection_name = 'requirments_collection',
        persist_directory = './requirments_collection'
    )
else:
    print("⚙️ Chroma DB not found. Creating new DB...")

    # ① 문서 로드
    loader = TextLoader("docs.txt", encoding="utf-8")
    documents = loader.load()

    # ② 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
    )
    split_docs = text_splitter.split_documents(documents)

    # ③ Chroma DB 생성 및 persist
    vectorstore = Chroma.from_documents(
        split_docs,
        embedding=embeddings,
        persist_directory='./requirments_collection'
    )
    print("✅ Chroma DB created and persisted.")

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
#from utils.config import get_llm
from workflow.state import AgentState
from langchain import hub
from typing import Literal
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command
from bs4 import BeautifulSoup


AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")

embeddings = AzureOpenAIEmbeddings(
    model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,
    azure_endpoint=AOAI_ENDPOINT
    )

if os.path.exists('./income_tax_collection') and len(os.listdir('./income_tax_collection')) > 0:
    vector_store = Chroma(
        embedding_function=embeddings,
        collection_name = 'income_tax_collection',
        persist_directory = './income_tax_collection'
    )
else:
    print("⚙️ Chroma DB not found. Creating new DB...")

    # ① 문서 로드
    loader = TextLoader("example.txt", encoding="utf-8")
    documents = loader.load()

    # ② 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
    )
    split_docs = text_splitter.split_documents(documents)

    # ③ Chroma DB 생성 및 persist
    vectorstore = Chroma.from_documents(
        split_docs,
        embedding=embeddings,
        persist_directory='./income_tax_collection'
    )
    print("✅ Chroma DB created and persisted.")

retriever = vector_store.as_retriever(search_kwargs={'k': 3})

# RAG 프롬프트를 가져옵니다.
generate_prompt = hub.pull("rlm/rag-prompt")

# .env 파일에서 환경 변수 로드
load_dotenv()

# LangChain Azure OpenAI 설정
llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AOAI_API_KEY"),
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    azure_deployment=os.getenv("AOAI_DEPLOY_GPT4O"),
    api_version=os.getenv("AOAI_API_VERSION"),
    temperature=0.7,
)




def retrieve(state: AgentState) -> AgentState:
    """
    사용자의 질문에 기반하여 벡터 스토어에서 관련 문서를 검색합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 검색된 문서가 추가된 state를 반환합니다.
    """
    query = state.query  # state에서 사용자의 질문을 추출합니다.
    docs = retriever.invoke(query)  # 질문과 관련된 문서를 검색합니다.
    #return {'context': docs}  # 검색된 문서를 포함한 state를 반환합니다.

    return AgentState(
            query=state.query,
            context=docs,
            rewrite_count=state.rewrite_count
        )

def generate(state: AgentState) -> AgentState:
    """
    주어진 state를 기반으로 RAG 체인을 사용하여 응답을 생성합니다.

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 생성된 응답을 포함하는 state를 반환합니다.
    """
    context = state.context  # state에서 문맥을 추출합니다.
    query = state.query      # state에서 사용자의 질문을 추출합니다.
    
    # RAG 체인을 구성합니다.
    rag_chain = generate_prompt | llm
    
    # 질문과 문맥을 사용하여 응답을 생성합니다.
    response = rag_chain.invoke({'question': query, 'context': context})
    
    #return {'answer': response}  # 생성된 응답을 포함하는 state를 반환합니다.

    return AgentState(
        query=state.query,
        context=state.context,
        answer=response.content,
        rewrite_count=state.rewrite_count
    )



# 문서 관련성 판단을 위한 프롬프트를 가져옵니다.
doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevance(state: AgentState) -> Literal['generate', 'rewrite']:
    """
    주어진 state를 기반으로 문서의 관련성을 판단합니다.

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        Literal['generate', 'rewrite']: 문서가 관련성이 높으면 'generate', 그렇지 않으면 'rewrite'를 반환합니다.
    """
    query = state.query  # state에서 사용자의 질문을 추출합니다.
    context = state.context  # state에서 문맥을 추출합니다.

    # 문서 관련성 판단 체인을 구성합니다.
    doc_relevance_chain = doc_relevance_prompt | llm
    
    # 질문과 문맥을 사용하여 문서의 관련성을 판단합니다.
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})

    if state.rewrite_count >= 3:
        return "generate"
        
    # 관련성이 높으면 'generate'를 반환하고, 그렇지 않으면 'rewrite'를 반환합니다.
    if response['Score'] == 1:
        return 'generate'
    else:
        state.rewrite_count += 1
        return 'rewrite'




# 사전 정의: 특정 표현을 다른 표현으로 변환하기 위한 사전입니다.
dictionary = ['사람과 관련된 표현 -> 거주자']

# 프롬프트 템플릿을 생성합니다. 사용자의 질문을 사전을 참고하여 변경합니다.
rewrite_prompt = PromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요 
사전: {dictionary}                                           
질문: {{query}}
""")

def rewrite(state: AgentState) -> AgentState:
    """
    사용자의 질문을 사전을 참고하여 변경합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 변경된 질문을 포함하는 state를 반환합니다.
    """
    query = state.query  # state에서 사용자의 질문을 추출합니다.
    
    # 리라이트 체인을 구성합니다. 프롬프트, LLM, 출력 파서를 연결합니다.
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()

    # 질문을 변경합니다.
    response = rewrite_chain.invoke({'query': query})
    
    #return {'query': response}  # 변경된 질문을 포함하는 state를 반환합니다.

    return AgentState(
        query=response,
        context=state.context,
        answer=state.answer,
        rewrite_count=state.rewrite_count
    )



@tool
def web_search(query: str) -> str:
    """
    DuckDuckGo를 이용한 간단한 웹 검색.
    """
    url = f"https://html.duckduckgo.com/html/?q={query}"
    try:
        resp = requests.get(url, timeout=5)
        soup = BeautifulSoup(resp.text, "html.parser")
        results = soup.find_all("a", class_="result__a")
        snippets = [a.text for a in results[:3]]
        return "\n".join(snippets) if snippets else "검색 결과가 없습니다."
    except Exception as e:
        print(e)
        return "검색 중 오류가 발생했습니다."

# 추가 한거
web_agent = create_react_agent(
    llm,
    tools=[web_search],
    prompt="당신은 최신 정보를 찾기 위해 웹 검색을 수행하는 웹봇입니다. 사용자 질문을 검색해서 답변하세요."
)

def web_node(state: AgentState) -> AgentState:
    # DuckDuckGo tool 호출
    search_result = web_search(state.query)

    web_prompt = f"""
    당신은 최신 정보를 찾기 위해 웹 검색을 수행하는 웹봇입니다.
    아래 검색 결과를 참고하여 사용자 질문에 답변하세요.

    검색결과:
    {search_result}

    질문: {state.query}
    """

    response = llm.invoke([HumanMessage(content=web_prompt)])
    text = getattr(response, "content", str(response))

    return AgentState(
        query=state.query,
        context=state.context,
        answer=text,
        rewrite_count=state.rewrite_count
    )


#def web_node(state: AgentState) -> AgentState:
#    result = web_agent.invoke(state)
#    return Command(
#        update={
#            "messages": [
#                HumanMessage(
#                    content=result["messages"][-1].content,
#                    name="web_search"
#                )
#            ]
#        },
#        goto="supervisor",
#    )